# TLT Medical Imaging Segmentation

This notebook includes 2 demos: 
1. training based on BraTS2018 data from scratch 
2. fine-tuning from a pretrained model based on your own data.

## Prerequisites 
1. you should have started this docker container with your own folder `brdemo` folder mounted to /mnt/brdemo
e.g.

docker run --runtime=nvidia -it --rm -v /your_local_path/brdemo:/mnt/brdemo -p 8888:8888 nvcr.io/nvtltea/med/tlt-aiaa:ea jupyter notebook /opt/nvidia/medical/segmentation/examples/brats --ip 0.0.0.0 --allow-root --no-browser

2. download Brats2018 training data into the brdemo/brats2018challeng folder, so that it's avaiable within docker at /mnt/brdemo/brats2018challenge

3. (for demo2) download/pull the pretrained brats model "segmentation_brain_br16_t1c2tc_v1" and place it in /mnt/brdemo/pretrained/segmentation_brain_br16_t1c2tc_v1 folder

FYI: this notebook file and the other config files are located in /opt/nvidia/medical/segmentation/examples/brats folder inside of the docker container.



# Demo 1: training on BraTS 2018 data (from scratch)

- Input: 4 channel 3D MRIs (T1c, T1, T2, FLAIR)
- Output: 3 channels of tumor subregions (tumor core, whole tumor, enhancing tumor)

## Define paths to config files and data
First, we need to define paths to configuration files and data. It's also possible to supply these paths directly to scripts, but we'll define them as variables for convenience.

In [8]:
# json config with a list of data files (brats2018 data in this case)
%env DATA_LIST=config_brats18_datalist1.json
# output directory to save outputs (such as Tensorboard logs and saved graph/checkpoints)
%env TRAIN_OUTPUT_DIR=/mnt/brdemo/output
# json config with training configureation
%env TRAIN_CONFIG=config_brats_8.json
## alternatively, if your GPU memory is >=16 GB, try larger (better) network config
## %env TRAIN_CONFIG=config_brats_16.json


env: DATA_LIST=config_brats18_datalist1.json
env: TRAIN_OUTPUT_DIR=/mnt/brdemo/output
env: TRAIN_CONFIG=config_brats_8.json


In [11]:
# Add number of gpus
%env NUM_GPU=8

env: NUM_GPU=8


In [14]:
!echo $NUM_GPU

8


### Data

We use the BraTS 2018 challenge dataset for the tutorial. To find more details please visit https://www.med.upenn.edu/sbia/brats2018/data.html. BraTS 2018 challenge dataset is already in Nifty format with isotrophic 1x1x1 mm pixel spacing. So no data conversion is needed. 

A json file `$DATA_LIST` lists the training and validation data.The data json file looks like:

In [2]:
!cat $DATA_LIST

{
    "training": [
        {
            "label": "training/HGG/Brats18_2013_2_1/Brats18_2013_2_1_seg.nii.gz",
            "image": [
                "training/HGG/Brats18_2013_2_1/Brats18_2013_2_1_t1ce.nii.gz",
                "training/HGG/Brats18_2013_2_1/Brats18_2013_2_1_t1.nii.gz",
                "training/HGG/Brats18_2013_2_1/Brats18_2013_2_1_t2.nii.gz",
                "training/HGG/Brats18_2013_2_1/Brats18_2013_2_1_flair.nii.gz"
            ]
        },
        {
            "label": "training/HGG/Brats18_2013_4_1/Brats18_2013_4_1_seg.nii.gz",
            "image": [
                "training/HGG/Brats18_2013_4_1/Brats18_2013_4_1_t1ce.nii.gz",
                "training/HGG/Brats18_2013_4_1/Brats18_2013_4_1_t1.nii.gz",
                "training/HGG/Brats18_2013_4_1/Brats18_2013_4_1_t2.nii.gz",
                "training/HGG/Brats18_2013_4_1/Brats18_2013_4_1_flair.nii.gz"
            ]
        },
        {
            "label": "training/HGG/Brats18_2013_7_1/Brats18_2013_7_1_seg.n

### Configuration

The json config file `$CONFIG_FILE` specifies the training options, including data I/O transforms, data augmentation transforms, model, loss, optimizer and hyper parameters. The training config file looks like:

In [9]:
!cat $TRAIN_CONFIG

{
    "description" : "Example of Brats segmentation. 3 outputs after sigmoid for 3 nested classes (TC,WT,ET).",
    "image_base_dir" : "/mnt/brdemo/brats2018challenge/",
    "num_channels": 4,
    "num_label_channels": 3,
    "num_classes": 3,

    "batch_size": 1,
    "epochs": 200,
    "num_workers": 8,
    "network_input_size": [224, 224, 128],
    "num_training_epoch_per_valid": 1,

    "record_step" : "every_epoch",
    "use_scanning_window" : false,
    "continue_from_checkpoint_epoch": true,
    "multi_gpu": false,

    "net_config":
	    {
		"name": "SegmResnet",
		"use_vae": false,
		"blocks_down": "1,2,2,4",
		"blocks_up": "1,1,1",
		"init_filters": 8,
		"use_groupnorm": true,
		"use_groupnormG": 8,
		"reg_weight": 1e-5,
		"dropout_prob": 0.2,
		"final_activation": "sigmoid"

	    },

    "auxiliary_outputs":
	    [
		{
		    "name": "common.metrics.metrics.dice_metric_masked_output",
		    "is_onehot_targets": true,
		    "is_independent_predictions": true,
		    "tags": ["

## 2. Run training

With data index file `$DATA_LIST` and `$CONFIG_FILE`, the `tlt-train` command can be used to run segmentation training and saves the outputs (including tensorboard log and checkpoint) in to the `$OUTPUT_DIR`:



In [10]:
!tlt-train segmentation \
    -e $TRAIN_CONFIG \
    -d $DATA_LIST \
    -r $TRAIN_OUTPUT_DIR

--------------------------------------------------------------------------
[[44602,1],0]: A high-performance Open MPI point-to-point messaging module
was unable to find any relevant network interfaces:

Module: OpenFabrics (openib)
  Host: 919f38d6be2b

Another transport will be used instead, although this may result in
lower performance.

NOTE: You can disable this warning by setting the MCA parameter
btl_base_warn_component_unused to 0.
--------------------------------------------------------------------------
Constructing transform: medical.common.transforms.transforms.LoadNifty
Constructing transform: medical.common.transforms.transforms.BratsConvertLabels
Constructing transform: medical.common.transforms.transforms.CropSubVolumeRandomWithinBounds
Constructing transform: medical.common.transforms.transforms.FlipAxisRandom
Constructing transform: medical.common.transforms.transforms.NormalizeNonzeroIntensities
Constructing transform: medical.common.transforms.transforms.AugmentInten

Constructing metric: MetricAverage with tag: mean_dice
Constructing metric: MetricAverage with tag: mean_dice_tc
Constructing metric: MetricAverage with tag: mean_dice_wt
Constructing metric: MetricAverage with tag: mean_dice_et
Constructing LR policy: medical.common.policy.lr_policy.ReduceLR_Poly
Starting Tensorflow fitting logDir: /mnt/brdemo/output epochs: 200 running_in_ngc: False
{'val_init_op': <tf.Operation 'make_initializer_1' type=MakeIterator>, 'tf_operations': <function create_vanilla_tensor_operation.<locals>.val_tensor_fn at 0x7fb7301b22f0>, 'metrics': [<medical.common.metrics.val_classes.MetricAverage object at 0x7fb79811e8d0>, <medical.common.metrics.val_classes.MetricAverage object at 0x7fb79811e6a0>, <medical.common.metrics.val_classes.MetricAverage object at 0x7fb79811eeb8>, <medical.common.metrics.val_classes.MetricAverage object at 0x7fb79811ef98>]}
CLEAN START (global_variables_initializer)
Total requested train epochs: 200 iters: 243 previous global epoch 0
Epoch:

In [16]:
!mpirun -np $NUM_GPU -H localhost:$NUM_GPU -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x LD_LIBRARY_PATH -x PATH -mca pml ob1 -mca btl ^openib --allow-run-as-root tlt-train segmentation \
    -e $TRAIN_CONFIG \
    -d $DATA_LIST \
    -r $TRAIN_OUTPUT_DIR

Constructing transform: medical.common.transforms.transforms.LoadNifty
Constructing transform: medical.common.transforms.transforms.BratsConvertLabels
Constructing transform: medical.common.transforms.transforms.CropSubVolumeRandomWithinBounds
Constructing transform: medical.common.transforms.transforms.FlipAxisRandom
Constructing transform: medical.common.transforms.transforms.LoadNifty
Constructing transform: medical.common.transforms.transforms.BratsConvertLabels
Constructing transform: medical.common.transforms.transforms.LoadNifty
Constructing transform: medical.common.transforms.transforms.BratsConvertLabels
Constructing transform: medical.common.transforms.transforms.CropSubVolumeRandomWithinBounds
Constructing transform: medical.common.transforms.transforms.NormalizeNonzeroIntensities
Constructing transform: medical.common.transforms.transforms.AugmentIntensityRandomScaleShift
Constructing transform: medical.common.transforms.transforms.LoadNifty
Constructing transform: medical

2019-05-15 19:44:15.002920: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-05-15 19:44:15.002983: I tensorflow/core/common_runtime/gpu/gpu_device.cc:988]      3 
2019-05-15 19:44:15.002993: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1001] 3:   N 
2019-05-15 19:44:15.003527: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14846 MB memory) -> physical GPU (device: 3, name: Tesla V100-SXM2-16GB, pci bus id: 0000:0b:00.0, compute capability: 7.0)
2019-05-15 19:44:15.024943: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-05-15 19:44:15.024998: I tensorflow/core/common_runtime/gpu/gpu_device.cc:988]      2 
2019-05-15 19:44:15.025024: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1001] 2:   N 
2019-05-15 19:44:15.025508: I tensorflow/core/co

LayerDown [ 0 : 0 ] shape [None, 8, 224, 224, 128]
LayerDown [ 1 : 0 ] shape [None, 16, 112, 112, 64]
LayerDown [ 1 : 1 ] shape [None, 16, 112, 112, 64]
LayerDown [ 0 : 0 ] shape [None, 8, 224, 224, 128]
LayerDown [ 2 : 0 ] shape [None, 32, 56, 56, 32]
LayerDown [ 1 : 0 ] shape [None, 16, 112, 112, 64]
LayerDown [ 0 : 0 ] shape [None, 8, 224, 224, 128]
LayerDown [ 0 : 0 ] shape [None, 8, 224, 224, 128]
LayerDown [ 0 : 0 ] shape [None, 8, 224, 224, 128]
LayerDown [ 2 : 1 ] shape [None, 32, 56, 56, 32]
LayerDown [ 1 : 1 ] shape [None, 16, 112, 112, 64]
LayerDown [ 0 : 0 ] shape [None, 8, 224, 224, 128]
LayerDown [ 1 : 0 ] shape [None, 16, 112, 112, 64]
LayerDown [ 0 : 0 ] shape [None, 8, 224, 224, 128]
LayerDown [ 0 : 0 ] shape [None, 8, 224, 224, 128]
LayerDown [ 3 : 0 ] shape [None, 64, 28, 28, 16]
LayerDown [ 1 : 0 ] shape [None, 16, 112, 112, 64]
LayerDown [ 1 : 0 ] shape [None, 16, 112, 112, 64]
LayerDown [ 2 : 0 ] shape [None, 32, 56, 56, 32]
LayerDown [ 1 : 1 ] shape [None, 16, 11

Constructing loss: losses.dice_loss
dice_loss targets [None, 3, 224, 224, 128] predictions [None, 3, 224, 224, 128] targets.dtype <dtype: 'float32'> predictions.dtype <dtype: 'float32'>
dice_loss is_channels_first: True skip_background: False is_onehot_targets True
Constructing loss: common.metrics.metrics.dice_metric_masked_output
metric_dice targets (?, 3, 224, 224, 128) predictions (?, 3, 224, 224, 128) targets.dtype <dtype: 'float32'> predictions.dtype <dtype: 'float32'>
metric_dice is_channels_first: True skip_background: False is_independent_predictions: True is_onehot_targets True threshold 0.5
Constructing loss: common.metrics.metrics.dice_metric_masked_output
metric_dice targets (?, 3, 224, 224, 128) predictions (?, 3, 224, 224, 128) targets.dtype <dtype: 'float32'> predictions.dtype <dtype: 'float32'>
metric_dice is_channels_first: True skip_background: False is_independent_predictions: True is_onehot_targets True threshold 0.5
Constructing metric: MetricAverage with tag: mea

Total requested train epochs: 200 iters: 31 previous global epoch 0
Total requested train epochs: 200 iters: 31 previous global epoch 0
Total requested train epochs: 200 iters: 31 previous global epoch 0
Total requested train epochs: 200 iters: 31 previous global epoch 0
Total requested train epochs: 200 iters: 31 previous global epoch 0
Total requested train epochs: 200 iters: 31 previous global epoch 0
Total requested train epochs: 200 iters: 31 previous global epoch 0
Total requested train epochs: 200 iters: 31 previous global epoch 0
919f38d6be2b:1177:1751 [0] NCCL INFO NET : Using interface eth0:172.17.0.5<0>
919f38d6be2b:1177:1751 [0] NCCL INFO NET/IB : Using interface eth0 for sideband communication
919f38d6be2b:1177:1751 [0] NCCL INFO Using internal Network Socket
919f38d6be2b:1177:1751 [0] NCCL INFO NET : Using interface eth0:172.17.0.5<0>
919f38d6be2b:1177:1751 [0] NCCL INFO NET/Socket : 1 interfaces found
NCCL version 2.3.7+cuda10.0
919f38d6be2b:1177:1751 [0] NCCL INFO rank 

919f38d6be2b:1193:1752 [4] NCCL INFO Ring 02 : 4[4] -> 7[7] via P2P/IPC
919f38d6be2b:1195:1754 [5] NCCL INFO Ring 02 : 5[5] -> 4[4] via P2P/IPC
919f38d6be2b:1196:1757 [6] NCCL INFO Ring 02 : 6[6] -> 2[2] via P2P/IPC
919f38d6be2b:1197:1750 [7] NCCL INFO Ring 02 : 7[7] -> 6[6] via P2P/IPC
919f38d6be2b:1184:1756 [2] NCCL INFO Ring 02 : 2[2] -> 0[0] via P2P/IPC
919f38d6be2b:1189:1755 [3] NCCL INFO Ring 02 : 3[3] -> 1[1] via P2P/IPC
919f38d6be2b:1177:1751 [0] NCCL INFO Ring 02 : 0[0] -> 3[3] via P2P/IPC
919f38d6be2b:1182:1753 [1] NCCL INFO Ring 02 : 1[1] -> 5[5] via P2P/IPC
919f38d6be2b:1184:1756 [2] NCCL INFO Ring 03 : 2[2] -> 1[1] via P2P/IPC
919f38d6be2b:1189:1755 [3] NCCL INFO Ring 03 : 3[3] -> 2[2] via P2P/IPC
919f38d6be2b:1182:1753 [1] NCCL INFO Ring 03 : 1[1] -> 5[5] via P2P/IPC
919f38d6be2b:1193:1752 [4] NCCL INFO Ring 03 : 4[4] -> 0[0] via P2P/IPC
919f38d6be2b:1195:1754 [5] NCCL INFO Ring 03 : 5[5] -> 6[6] via P2P/IPC
919f38d6be2b:1196:1757 [6] NCCL INFO Ring 03 : 6[6] -> 7[7] via 

Epoch: 6/200, train_dice_et: 0.1410  train_loss: 0.9089  train_dice_wt: 0.5371  train_dice_tc: 0.2482  train_dice: 0.3087  mean_dice_tc: 0.3601  mean_dice_wt: 0.6628  mean_dice: 0.4126  mean_dice_et: 0.2094  val_time: 5.54s    
New best val metric: 0.41259026527404785. Saving model
Saving model checkpoint at location: /mnt/brdemo/output/model.ckpt
Epoch: 7/200, train_dice_et: 0.0514  train_loss: 0.8983  train_dice_wt: 0.5557  train_dice_tc: 0.2124  train_dice: 0.2732  mean_dice_tc: 0.2822  mean_dice_wt: 0.6782  mean_dice: 0.3376  mean_dice_et: 0.0291  val_time: 4.92s    
Epoch: 8/200, train_dice_et: 0.0196  train_loss: 0.8779  train_dice_wt: 0.6091  train_dice_tc: 0.2731  train_dice: 0.3006  mean_dice_tc: 0.3896  mean_dice_wt: 0.7002  mean_dice: 0.3838  mean_dice_et: 0.0310  val_time: 4.94s    
Epoch: 9/200, train_dice_et: 0.0212  train_loss: 0.8453  train_dice_wt: 0.6559  train_dice_tc: 0.3316  train_dice: 0.3362  mean_dice_tc: 0.4409  mean_dice_wt: 0.7585  mean_dice: 0.4250  mean_dic

Epoch: 64/200, train_dice_et: 0.6944  train_loss: 0.1760  train_dice_wt: 0.8688  train_dice_tc: 0.8090  train_dice: 0.7907  mean_dice_tc: 0.7973  mean_dice_wt: 0.8666  mean_dice: 0.7880  mean_dice_et: 0.6978  val_time: 5.14s    
Epoch: 65/200, train_dice_et: 0.6841  train_loss: 0.1804  train_dice_wt: 0.8664  train_dice_tc: 0.8129  train_dice: 0.7878  mean_dice_tc: 0.8011  mean_dice_wt: 0.8749  mean_dice: 0.7965  mean_dice_et: 0.7107  val_time: 4.97s    
Epoch: 66/200, train_dice_et: 0.7118  train_loss: 0.1598  train_dice_wt: 0.8742  train_dice_tc: 0.8388  train_dice: 0.8083  mean_dice_tc: 0.8089  mean_dice_wt: 0.8688  mean_dice: 0.7981  mean_dice_et: 0.7110  val_time: 4.97s    
Epoch: 67/200, train_dice_et: 0.7027  train_loss: 0.1793  train_dice_wt: 0.8440  train_dice_tc: 0.8216  train_dice: 0.7894  mean_dice_tc: 0.8086  mean_dice_wt: 0.8735  mean_dice: 0.7972  mean_dice_et: 0.7039  val_time: 5.02s    
Epoch: 68/200, train_dice_et: 0.7082  train_loss: 0.1716  train_dice_wt: 0.8622  tra

Epoch: 130/200, train_dice_et: 0.7627  train_loss: 0.1323  train_dice_wt: 0.8891  train_dice_tc: 0.8631  train_dice: 0.8383  mean_dice_tc: 0.8271  mean_dice_wt: 0.8825  mean_dice: 0.8237  mean_dice_et: 0.7599  val_time: 5.19s    
Epoch: 131/200, train_dice_et: 0.7604  train_loss: 0.1310  train_dice_wt: 0.8934  train_dice_tc: 0.8667  train_dice: 0.8401  mean_dice_tc: 0.8271  mean_dice_wt: 0.8827  mean_dice: 0.8218  mean_dice_et: 0.7554  val_time: 4.96s    
Epoch: 132/200, train_dice_et: 0.7524  train_loss: 0.1359  train_dice_wt: 0.8918  train_dice_tc: 0.8577  train_dice: 0.8340  mean_dice_tc: 0.8347  mean_dice_wt: 0.8836  mean_dice: 0.8264  mean_dice_et: 0.7602  val_time: 4.87s    
Epoch: 133/200, train_dice_et: 0.7545  train_loss: 0.1370  train_dice_wt: 0.8848  train_dice_tc: 0.8631  train_dice: 0.8341  mean_dice_tc: 0.8222  mean_dice_wt: 0.8845  mean_dice: 0.8172  mean_dice_et: 0.7424  val_time: 5.04s    
Epoch: 134/200, train_dice_et: 0.7483  train_loss: 0.1414  train_dice_wt: 0.8815

Saving model checkpoint at location: /mnt/brdemo/output/model.ckpt
Epoch: 164/200, train_dice_et: 0.7815  train_loss: 0.1244  train_dice_wt: 0.8931  train_dice_tc: 0.8694  train_dice: 0.8480  mean_dice_tc: 0.8253  mean_dice_wt: 0.8873  mean_dice: 0.8238  mean_dice_et: 0.7579  val_time: 4.91s    
Epoch: 165/200, train_dice_et: 0.7702  train_loss: 0.1287  train_dice_wt: 0.8851  train_dice_tc: 0.8750  train_dice: 0.8435  mean_dice_tc: 0.8303  mean_dice_wt: 0.8868  mean_dice: 0.8264  mean_dice_et: 0.7610  val_time: 4.86s    
Epoch: 166/200, train_dice_et: 0.7691  train_loss: 0.1270  train_dice_wt: 0.8924  train_dice_tc: 0.8729  train_dice: 0.8448  mean_dice_tc: 0.8284  mean_dice_wt: 0.8865  mean_dice: 0.8253  mean_dice_et: 0.7602  val_time: 4.92s    
Epoch: 167/200, train_dice_et: 0.7636  train_loss: 0.1333  train_dice_wt: 0.8898  train_dice_tc: 0.8591  train_dice: 0.8375  mean_dice_tc: 0.8265  mean_dice_wt: 0.8905  mean_dice: 0.8233  mean_dice_et: 0.7518  val_time: 5.15s    
Epoch: 168/20

In [30]:
! ls -lt /mnt/brdemo/output/

total 60808
-rw-r--r-- 1 root root    10361 May 15 22:13 mean_dice_ET_raw_results.txt
-rw-r--r-- 1 root root      132 May 15 22:13 mean_dice_ET_summary_results.txt
-rw-r--r-- 1 root root    11514 May 15 22:13 mean_dice_TC_raw_results.txt
-rw-r--r-- 1 root root      132 May 15 22:13 mean_dice_TC_summary_results.txt
-rw-r--r-- 1 root root    11515 May 15 22:13 mean_dice_WT_raw_results.txt
-rw-r--r-- 1 root root      132 May 15 22:13 mean_dice_WT_summary_results.txt
-rw-r--r-- 1 root root  4967421 May 15 21:49 model.trt.pb
-rw-r--r-- 1 root root  5021100 May 15 21:49 model.fzn.pb
-rw-r--r-- 1 root root  2184895 May 15 21:46 model_final.ckpt.meta
-rw-r--r-- 1 root root      130 May 15 21:46 checkpoint
-rw-r--r-- 1 root root     9874 May 15 21:46 model_final.ckpt.index
-rw-r--r-- 1 root root 14122132 May 15 21:46 model_final.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root  4213513 May 15 21:46 events.out.tfevents.1557949467.919f38d6be2b
-rw-r--r-- 1 root root  2184895 May 15 21:37 model.ckp

In [ ]:
# !tlt-evaluate \
#     --model_save_path $MODEL_SAVE_PATH \
#     --config $CONFIG \
#     --model_format $MODEL_FORMAT \
#     --file_root $DATA_ROOT \
#     --output_path $OUTPUT_PATH \
#     --data_file $DATA_FILE \
#     --setup scanning_window \
#     --no_overwrite

## 3. Evaluate trained model

After trainining is done, we use `tlt-export` command to freeze the checkpoint.

In [20]:
%env CHECKPOINT=/mnt/brdemo/output

# export the frozen model
!tlt-export \
    --model_name 'model' \
    --model_file_path ${CHECKPOINT} \
    --input_node_names 'NV_INPUTALL_ITERATORGETNEXT' \
    --output_node_names 'NV_OUTPUT_LABEL'

env: CHECKPOINT=/mnt/brdemo/output
Creating a regular frozen graph from Checkpoint at '/mnt/brdemo/output' ...
Instructions for updating:
Use tf.gfile.GFile.
Loaded meta graph file '/mnt/brdemo/output/model.ckpt.meta
2019-05-15 21:49:32.671104: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: Tesla V100-SXM2-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.53
pciBusID: 0000:06:00.0
totalMemory: 15.75GiB freeMemory: 15.34GiB
2019-05-15 21:49:33.035193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 1 with properties: 
name: Tesla V100-SXM2-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.53
pciBusID: 0000:07:00.0
totalMemory: 15.75GiB freeMemory: 15.34GiB
2019-05-15 21:49:33.401788: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 2 with properties: 
name: Tesla V100-SXM2-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.53
pciBusID: 0000:0a:00.0
totalMemory: 15.75GiB freeMemory: 15.34GiB
2019-05-15 21:4

2019-05-15 21:49:43.618599: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:501] Optimization results for grappler item: tf_graph
2019-05-15 21:49:43.618641: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:503]   constant folding: Graph size after: 756 nodes (-133), 861 edges (-134), time = 46.087ms.
2019-05-15 21:49:43.618664: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:503]   layout: Graph size after: 756 nodes (0), 861 edges (0), time = 16.268ms.
2019-05-15 21:49:43.618670: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:503]   TensorRTOptimizer: Graph size after: 756 nodes (0), 861 edges (0), time = 35.138ms.
2019-05-15 21:49:43.618676: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:503]   constant folding: Graph size after: 756 nodes (0), 861 edges (0), time = 19.529ms.
2019-05-15 21:49:43.618682: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:503]   TensorRTOptimizer: Graph size after: 756 nodes (0), 861 edges (0), time = 35.4

Once the training finishes and trained model is frozen, `tlt-evaluate` command can be used to run evaluation of the trained model. Evaluation requires its own config file, corresponding to the training configuration file. It looks like:

In [21]:
%env CONFIG=config_validation.json
%env DATA_ROOT=/mnt/brdemo/brats2018challenge/
%env DATA_FILE=config_brats18_datalist1.json
%env MODEL_FORMAT=frozenmodel
%env MODEL_SAVE_PATH=/mnt/brdemo/output/model.trt.pb
%env OUTPUT_PATH=/mnt/brdemo/output

!mkdir $OUTPUT_PATH
!cat $CONFIG

env: CONFIG=config_validation.json
env: DATA_ROOT=/mnt/brdemo/brats2018challenge/
env: DATA_FILE=config_brats18_datalist1.json
env: MODEL_FORMAT=frozenmodel
env: MODEL_SAVE_PATH=/mnt/brdemo/output/model.trt.pb
env: OUTPUT_PATH=/mnt/brdemo/output
mkdir: cannot create directory ‘/mnt/brdemo/output’: File exists
{

    "input_nodes":
    {
        "image": "NV_INPUTALL_ITERATORGETNEXT"
    },
    "output_nodes":
    {
        "model": "NV_OUTPUT_LABEL"
    },
    "batch_size": 1,
    "pre_transforms":
    [
        {
            "name": "transforms.LoadNifty",
            "fields": "image"
        },
        {
            "name": "transforms.NormalizeNonzeroIntensities",
            "fields": "image"
        }
    ],
    "post_transforms":
    [
        {
            "name": "transforms.ThresholdValues",
            "applied_key": "model",
            "threshold": 0.5
        },
        {
            "name": "transforms.SplitAcrossChannels",
            "applied_key": "model",
           

Now we can run evaluation (evaluation runs the trained checkpoint model, on files in your DATA_LIST validation section and compute the average dice per file per class. ground truth segmentation must be available)

In [29]:
!tlt-evaluate \
    --model_save_path $MODEL_SAVE_PATH \
    --config $CONFIG \
    --model_format $MODEL_FORMAT \
    --file_root $DATA_ROOT \
    --output_path $OUTPUT_PATH \
    --data_file $DATA_FILE \
    --setup scanning_window \
    --no_overwrite

Constructing transform: medical.common.transforms.transforms.LoadNifty
Constructing transform: medical.common.transforms.transforms.NormalizeNonzeroIntensities
Constructing transform: medical.common.transforms.transforms.ThresholdValues
Constructing transform: medical.common.transforms.transforms.SplitAcrossChannels
Constructing transform: medical.common.transforms.transforms.LoadNifty
Constructing transform: medical.common.transforms.transforms.BratsConvertLabels
Constructing transform: medical.common.transforms.transforms.SplitAcrossChannels
--------------------------------------------------------------------------
[[29493,1],0]: A high-performance Open MPI point-to-point messaging module
was unable to find any relevant network interfaces:

Module: OpenFabrics (openib)
  Host: 919f38d6be2b

Another transport will be used instead, although this may result in
lower performance.

NOTE: You can disable this warning by setting the MCA parameter
btl_base_warn_component_unused to 0.
-------

['training/HGG/Brats18_2013_20_1/Brats18_2013_20_1_t1ce.nii.gz', 'training/HGG/Brats18_2013_20_1/Brats18_2013_20_1_t1.nii.gz', 'training/HGG/Brats18_2013_20_1/Brats18_2013_20_1_t2.nii.gz', 'training/HGG/Brats18_2013_20_1/Brats18_2013_20_1_flair.nii.gz']
['training/HGG/Brats18_2013_22_1/Brats18_2013_22_1_t1ce.nii.gz', 'training/HGG/Brats18_2013_22_1/Brats18_2013_22_1_t1.nii.gz', 'training/HGG/Brats18_2013_22_1/Brats18_2013_22_1_t2.nii.gz', 'training/HGG/Brats18_2013_22_1/Brats18_2013_22_1_flair.nii.gz']
['training/HGG/Brats18_CBICA_AAB_1/Brats18_CBICA_AAB_1_t1ce.nii.gz', 'training/HGG/Brats18_CBICA_AAB_1/Brats18_CBICA_AAB_1_t1.nii.gz', 'training/HGG/Brats18_CBICA_AAB_1/Brats18_CBICA_AAB_1_t2.nii.gz', 'training/HGG/Brats18_CBICA_AAB_1/Brats18_CBICA_AAB_1_flair.nii.gz']
['training/HGG/Brats18_CBICA_ALN_1/Brats18_CBICA_ALN_1_t1ce.nii.gz', 'training/HGG/Brats18_CBICA_ALN_1/Brats18_CBICA_ALN_1_t1.nii.gz', 'training/HGG/Brats18_CBICA_ALN_1/Brats18_CBICA_ALN_1_t2.nii.gz', 'training/HGG/Brats18

['training/LGG/Brats18_TCIA10_393_1/Brats18_TCIA10_393_1_t1ce.nii.gz', 'training/LGG/Brats18_TCIA10_393_1/Brats18_TCIA10_393_1_t1.nii.gz', 'training/LGG/Brats18_TCIA10_393_1/Brats18_TCIA10_393_1_t2.nii.gz', 'training/LGG/Brats18_TCIA10_393_1/Brats18_TCIA10_393_1_flair.nii.gz']
['training/LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_t1ce.nii.gz', 'training/LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_t1.nii.gz', 'training/LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_t2.nii.gz', 'training/LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_flair.nii.gz']
['training/LGG/Brats18_TCIA12_470_1/Brats18_TCIA12_470_1_t1ce.nii.gz', 'training/LGG/Brats18_TCIA12_470_1/Brats18_TCIA12_470_1_t1.nii.gz', 'training/LGG/Brats18_TCIA12_470_1/Brats18_TCIA12_470_1_t2.nii.gz', 'training/LGG/Brats18_TCIA12_470_1/Brats18_TCIA12_470_1_flair.nii.gz']
['training/LGG/Brats18_TCIA13_623_1/Brats18_TCIA13_623_1_t1ce.nii.gz', 'training/LGG/Brats18_TCIA13_623_1/Brats18_TCIA13_623_1_t1.nii.gz', 'training/LGG/Brats18_TCIA1

This command generates an evaluation report at `$EVALUATION_REPORT`. It looks like:

In [34]:
!ls -lt /mnt/brdemo/output/

total 60808
-rw-r--r-- 1 root root    10361 May 15 22:13 mean_dice_ET_raw_results.txt
-rw-r--r-- 1 root root      132 May 15 22:13 mean_dice_ET_summary_results.txt
-rw-r--r-- 1 root root    11514 May 15 22:13 mean_dice_TC_raw_results.txt
-rw-r--r-- 1 root root      132 May 15 22:13 mean_dice_TC_summary_results.txt
-rw-r--r-- 1 root root    11515 May 15 22:13 mean_dice_WT_raw_results.txt
-rw-r--r-- 1 root root      132 May 15 22:13 mean_dice_WT_summary_results.txt
-rw-r--r-- 1 root root  4967421 May 15 21:49 model.trt.pb
-rw-r--r-- 1 root root  5021100 May 15 21:49 model.fzn.pb
-rw-r--r-- 1 root root  2184895 May 15 21:46 model_final.ckpt.meta
-rw-r--r-- 1 root root      130 May 15 21:46 checkpoint
-rw-r--r-- 1 root root     9874 May 15 21:46 model_final.ckpt.index
-rw-r--r-- 1 root root 14122132 May 15 21:46 model_final.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root  4213513 May 15 21:46 events.out.tfevents.1557949467.919f38d6be2b
-rw-r--r-- 1 root root  2184895 May 15 21:37 model.ckp

In [36]:
%env EVALUATION_REPORT=/mnt/brdemo/output/mean_dice_ET_summary_results.txt
!cat $EVALUATION_REPORT

env: EVALUATION_REPORT=/mnt/brdemo/output/mean_dice_ET_summary_results.txt
mean_dice_ET (statistics of 38 valid cases):
    mean  median     max     min   90percent
   0.770   0.837   0.927   0.000   0.668



In [37]:
%env EVALUATION_REPORT_RAW=/mnt/brdemo/output/mean_dice_ET_raw_results.txt
!cat $EVALUATION_REPORT_RAW

env: EVALUATION_REPORT_RAW=/mnt/brdemo/output/mean_dice_ET_raw_results.txt
training/HGG/Brats18_2013_3_1/Brats18_2013_3_1_t1ce.nii.gz,training/HGG/Brats18_2013_3_1/Brats18_2013_3_1_t1.nii.gz,training/HGG/Brats18_2013_3_1/Brats18_2013_3_1_t2.nii.gz,training/HGG/Brats18_2013_3_1/Brats18_2013_3_1_flair.nii.gz	0.838492566555261
training/HGG/Brats18_2013_5_1/Brats18_2013_5_1_t1ce.nii.gz,training/HGG/Brats18_2013_5_1/Brats18_2013_5_1_t1.nii.gz,training/HGG/Brats18_2013_5_1/Brats18_2013_5_1_t2.nii.gz,training/HGG/Brats18_2013_5_1/Brats18_2013_5_1_flair.nii.gz	0.8816389365859435
training/HGG/Brats18_2013_10_1/Brats18_2013_10_1_t1ce.nii.gz,training/HGG/Brats18_2013_10_1/Brats18_2013_10_1_t1.nii.gz,training/HGG/Brats18_2013_10_1/Brats18_2013_10_1_t2.nii.gz,training/HGG/Brats18_2013_10_1/Brats18_2013_10_1_flair.nii.gz	0.843084434716727
training/HGG/Brats18_2013_12_1/Brats18_2013_12_1_t1ce.nii.gz,training/HGG/Brats18_2013_12_1/Brats18_2013_12_1_t1.nii.gz,training/HGG/Brats18_2013_12_1/Brats18_2013

In [32]:
%env EVALUATION_REPORT=/mnt/brdemo/output/mean_dice_WT_summary_results.txt
!cat $EVALUATION_REPORT

env: EVALUATION_REPORT=/mnt/brdemo/output/mean_dice_WT_summary_results.txt
mean_dice_WT (statistics of 42 valid cases):
    mean  median     max     min   90percent
   0.892   0.913   0.962   0.523   0.843



In [33]:
%env EVALUATION_REPORT=/mnt/brdemo/output/mean_dice_TC_summary_results.txt
!cat $EVALUATION_REPORT

env: EVALUATION_REPORT=/mnt/brdemo/output/mean_dice_TC_summary_results.txt
mean_dice_TC (statistics of 42 valid cases):
    mean  median     max     min   90percent
   0.855   0.891   0.957   0.445   0.745



## 4. Run inference
You can also run inference of the model on your data using the inference command (it saves the output segmentation masks as nifty files, no ground truth is required)

In [38]:
%env CONFIG=config_validation.json
%env DATA_ROOT=/mnt/brdemo/brats2018challenge/
%env DATA_FILE=config_brats18_datalist1.json
%env MODEL_FORMAT=frozenmodel
%env MODEL_SAVE_PATH=/mnt/brdemo/output/model.trt.pb
%env OUTPUT_PATH=/mnt/brdemo/output


!mkdir $OUTPUT_PATH
!cat $CONFIG

!tlt-infer \
    --model_save_path $MODEL_SAVE_PATH \
    --config $CONFIG \
    --model_format $MODEL_FORMAT \
    --file_root $DATA_ROOT \
    --output_path $OUTPUT_PATH \
    --data_file $DATA_FILE \
    --setup scanning_window \
    --no_overwrite

env: CONFIG=config_validation.json
env: DATA_ROOT=/mnt/brdemo/brats2018challenge/
env: DATA_FILE=config_brats18_datalist1.json
env: MODEL_FORMAT=frozenmodel
env: MODEL_SAVE_PATH=/mnt/brdemo/output/model.trt.pb
env: OUTPUT_PATH=/mnt/brdemo/output
mkdir: cannot create directory ‘/mnt/brdemo/output’: File exists
{

    "input_nodes":
    {
        "image": "NV_INPUTALL_ITERATORGETNEXT"
    },
    "output_nodes":
    {
        "model": "NV_OUTPUT_LABEL"
    },
    "batch_size": 1,
    "pre_transforms":
    [
        {
            "name": "transforms.LoadNifty",
            "fields": "image"
        },
        {
            "name": "transforms.NormalizeNonzeroIntensities",
            "fields": "image"
        }
    ],
    "post_transforms":
    [
        {
            "name": "transforms.ThresholdValues",
            "applied_key": "model",
            "threshold": 0.5
        },
        {
            "name": "transforms.SplitAcrossChannels",
            "applied_key": "model",
           

['training/HGG/Brats18_2013_5_1/Brats18_2013_5_1_t1ce.nii.gz', 'training/HGG/Brats18_2013_5_1/Brats18_2013_5_1_t1.nii.gz', 'training/HGG/Brats18_2013_5_1/Brats18_2013_5_1_t2.nii.gz', 'training/HGG/Brats18_2013_5_1/Brats18_2013_5_1_flair.nii.gz']
['training/HGG/Brats18_2013_10_1/Brats18_2013_10_1_t1ce.nii.gz', 'training/HGG/Brats18_2013_10_1/Brats18_2013_10_1_t1.nii.gz', 'training/HGG/Brats18_2013_10_1/Brats18_2013_10_1_t2.nii.gz', 'training/HGG/Brats18_2013_10_1/Brats18_2013_10_1_flair.nii.gz']
['training/HGG/Brats18_2013_12_1/Brats18_2013_12_1_t1ce.nii.gz', 'training/HGG/Brats18_2013_12_1/Brats18_2013_12_1_t1.nii.gz', 'training/HGG/Brats18_2013_12_1/Brats18_2013_12_1_t2.nii.gz', 'training/HGG/Brats18_2013_12_1/Brats18_2013_12_1_flair.nii.gz']
['training/HGG/Brats18_2013_14_1/Brats18_2013_14_1_t1ce.nii.gz', 'training/HGG/Brats18_2013_14_1/Brats18_2013_14_1_t1.nii.gz', 'training/HGG/Brats18_2013_14_1/Brats18_2013_14_1_t2.nii.gz', 'training/HGG/Brats18_2013_14_1/Brats18_2013_14_1_flair.n

['training/LGG/Brats18_2013_15_1/Brats18_2013_15_1_t1ce.nii.gz', 'training/LGG/Brats18_2013_15_1/Brats18_2013_15_1_t1.nii.gz', 'training/LGG/Brats18_2013_15_1/Brats18_2013_15_1_t2.nii.gz', 'training/LGG/Brats18_2013_15_1/Brats18_2013_15_1_flair.nii.gz']
['training/LGG/Brats18_2013_24_1/Brats18_2013_24_1_t1ce.nii.gz', 'training/LGG/Brats18_2013_24_1/Brats18_2013_24_1_t1.nii.gz', 'training/LGG/Brats18_2013_24_1/Brats18_2013_24_1_t2.nii.gz', 'training/LGG/Brats18_2013_24_1/Brats18_2013_24_1_flair.nii.gz']
['training/LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_t1ce.nii.gz', 'training/LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_t1.nii.gz', 'training/LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_t2.nii.gz', 'training/LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_flair.nii.gz']
['training/LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_t1ce.nii.gz', 'training/LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_t1.nii.gz', 'training/LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_t2.nii.gz', 'traini

***
***
***

# Demo 2:  Fine-tuning a pre-trained model on a small set of new images

Pretrained model was trained with:
- Input: 1 channel MRIs (T1c only)
- Output: 1 channel(tumor core only)


First, set the API_KEY environment variable for later use.  To get your API key, login to your ngc account at ngc.nvidia.com. Then go to settings→Generate API key to generate your key. Note down the key in a secure location as this will be needed every time tlt-pull runs.

In [44]:
%env API_KEY=OWs4Z3BzYTEyZTQ4MnZzNW1hdGV2bDBubHU6ODkwNDMyZGEtMDBjNS00MjEwLWExM2UtNDNiY2JhOTRmNWFj

env: API_KEY=OWs4Z3BzYTEyZTQ4MnZzNW1hdGV2bDBubHU6ODkwNDMyZGEtMDBjNS00MjEwLWExM2UtNDNiY2JhOTRmNWFj


### Getting list of models
To view the list of available models, run

In [45]:
!tlt-pull -lm -k $API_KEY -o nvtltea -t med

Traceback (most recent call last):
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "medical/ngc/pull.py", line 78, in <module>
  File "medical/ngc/pull.py", line 64, in main
  File "/home/isaacy/workspace/repos/ai-infra/medical/ngc/controller.py", line 52, in list_models
  File "/usr/local/lib/python2.7/dist-packages/ngccli/controller/model.py", line 36, in list
    model_list_resp = self.model_api.list(org_name, team_name)
  File "/usr/local/lib/python2.7/dist-packages/ngccli/api/models.py", line 90, in list
    auth_team=team_name)
  File "/usr/local/lib/python2.7/dist-packages/ngccli/api/connection.py", line 155, in make_api_request
    return self._parse_api_response(response, url)
  File "/usr/local/lib/python2.7/dist-packages/ngccli/api/connection.py", line 270, in _parse_api_response
    o = self._result(response, is_json

### Getting versions of a model

In [46]:
!tlt-pull -k $API_KEY -lv -m segmentation_brain_br16_t1c2tc -o nvtltea -t med

Traceback (most recent call last):
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "medical/ngc/pull.py", line 78, in <module>
  File "medical/ngc/pull.py", line 68, in main
  File "/home/isaacy/workspace/repos/ai-infra/medical/ngc/controller.py", line 58, in list_versions
  File "/usr/local/lib/python2.7/dist-packages/ngccli/controller/model.py", line 165, in list_versions
    raise ResourceNotFoundException("Model '{}' could not be found.".format(model_name))
ngccli.errors.ResourceNotFoundException: Error: Model 'segmentation_brain_br16_t1c2tc' could not be found.


### Downloading a model
To download a model, run the following command.  Note -v is a mandatory argument. Please use -lv to find available versions.

In [ ]:
!tlt-pull -k $API_KEY -m segmentation_brain_br16_t1c2tc -v 1 -o nvtltea -t med -d /mnt/brdemo/pretrained/segmentation_brain_br16_t1c2tc

In [ ]:
## please create a datalist with your own data listing, and use it as DATA_LIST instead of config_brats18_datalist_t1c.json
# %env DATA_LIST=config_brats18_datalist_t1c.json
%env DATA_LIST=config_brats18_datalist1.json
#%env DATA_LIST=/yourfolder/yourdatalist.json

%env TRAIN_OUTPUT_DIR=/mnt/brdemo/output2
# %env TRAIN_CONFIG=config_brats_t1c2tc_f16_ft.json
%env TRAIN_CONFIG=config_brats_t1c2tc_f8_ft.json
%env CHECKPOINT=/mnt/brdemo/pretrained/segmentation_brain_br16_t1c2tc


### Data conversion

The pretrained model was trained on 1x1x1mm resolution nifty T1c images. If your data files are in different format/resolution, you can use the data converter provided in TLT to convert and resample your data. The following command converts all dicom volumes in `your/data/directory` to Nifty format and resamples them to isotrophic 1x1x1mm resolution. If the images to be converted are segmentation labels, then an option `-l` needs to be added, and the resampler will use nearest neighbor interpolator (otherwise linear interpolator is used.).

In [ ]:
#!tlt-dataconvert -d your/data/directory -r 1 -s .dcm -e .nii.gz -o your/output/directory

### Train/Finetune
Finetuning is similar to training, except we initialize the model from the pretrained weights (pretrained checkpoint)

In [ ]:
!tlt-train segmentation \
    -c $CHECKPOINT/model.ckpt \
    -e $TRAIN_CONFIG \
    -d $DATA_LIST \
    -r $TRAIN_OUTPUT_DIR

In [ ]:
!mpirun -np $NUM_GPU -H localhost:$NUM_GPU -bind-to none -map-by slot -x NCCL_DEBUG=INFO -x LD_LIBRARY_PATH -x PATH -mca pml ob1 -mca btl ^openib --allow-run-as-root tlt-train segmentation \
    -c $CHECKPOINT/model.ckpt \
    -e $TRAIN_CONFIG \
    -d $DATA_LIST \
    -r $TRAIN_OUTPUT_DIR

After trainining is done, we use `tlt-export` command to freeze the checkpoint.

In [ ]:
# if you didn't run the full retraining, you can just copy our pretraine model inside of TRAIN_OUTPUT_DIR (output2)
#cp -R /mnt/brdemo/pretrained/segmentation_brain_br16_t1c2tc /mnt/brdemo/output2

# export the frozen model
!tlt-export \
    --model_name 'model' \
    --model_file_path ${TRAIN_OUTPUT_DIR} \
    --input_node_names 'NV_INPUTALL_ITERATORGETNEXT' \
    --output_node_names 'NV_OUTPUT_LABEL'

Evaluate the finetuned model, notice we use `$TRAIN_OUTPUT_DIR` as a checkpoint dir, since this is where the finedtuned model was saved:

In [ ]:
%env CONFIG=config_validation_ft.json
%env DATA_ROOT=/mnt/brdemo/brats2018challenge
%env DATA_FILE=config_brats18_datalist_t1c.json
%env MODEL_FORMAT=frozenmodel
%env MODEL_SAVE_PATH=/mnt/brdemo/output2/model.trt.pb
%env OUTPUT_PATH=/mnt/brdemo/output2

!mkdir $OUTPUT_PATH
!cat $CONFIG

!tlt-evaluate \
    --model_save_path $MODEL_SAVE_PATH \
    --config $CONFIG \
    --model_format $MODEL_FORMAT \
    --file_root $DATA_ROOT \
    --output_path $OUTPUT_PATH \
    --data_file $DATA_FILE \
    --setup scanning_window \
    --no_overwrite

Same syntax can be used for inference (to save the output masks). Here we reuse DATA_FILE and CONFIG files (even though they include the info about ground truth labels, which is not necessary for inference).

In [ ]:
%env CONFIG=config_validation_ft.json
%env DATA_ROOT=/mnt/brdemo/brats2018challenge
%env DATA_FILE=config_brats18_datalist_t1c.json
%env MODEL_FORMAT=frozenmodel
%env MODEL_SAVE_PATH=/mnt/brdemo/output2/model.trt.pb
%env OUTPUT_PATH=/mnt/brdemo/output2

!mkdir $OUTPUT_PATH
!cat $CONFIG

!tlt-infer \
    --model_save_path $MODEL_SAVE_PATH \
    --config $CONFIG \
    --model_format $MODEL_FORMAT \
    --file_root $DATA_ROOT \
    --output_path $OUTPUT_PATH \
    --data_file $DATA_FILE \
    --setup scanning_window \
    --no_overwrite

***
***

# Visualize prediction results

After running inference, you can use the following python script by editing the `name` variable visualize the predicted segmentation mask:

In [ ]:
from visualization import display_prediction

name= 'Brats18_2013_3_1'

vol_fname = '/mnt/brdemo/brats2018challenge/training/HGG/'+name+'/'+name+'_t1ce.nii.gz'
pred_fname = '/mnt/brdemo/output2/inference_output/'+name+'_t1ce_prediction.nii.gz'
print("image: "+vol_fname)
print("prediction: "+pred_fname)

display_prediction(vol_fname, pred_fname)